In [3]:
from collections import namedtuple

In [4]:
Point = namedtuple("Point", ["x", "y"])
p = Point(1, 2)

a, b = p

In [5]:
a, b

(1, 2)

In [2]:
import threading

In [7]:
threading.active_count()

7

In [8]:
threading.enumerate()

[<_MainThread(MainThread, started 140426923098112)>,
 <Thread(IOPub, started daemon 140426857371200)>,
 <Heartbeat(Heartbeat, started daemon 140426774312512)>,
 <Thread(Thread-2 (_watch_pipe_fd), started daemon 140426749134400)>,
 <Thread(Thread-3 (_watch_pipe_fd), started daemon 140426740741696)>,
 <ControlThread(Control, started daemon 140426732348992)>,
 <HistorySavingThread(IPythonHistorySavingThread, started 140426723956288)>]

In [9]:
threading.current_thread()

<_MainThread(MainThread, started 140426923098112)>

In [10]:
th = threading.main_thread()
th.name, th.ident, th.native_id

('MainThread', 140426923098112, 4683)

In [11]:
threading.stack_size()

0

In [12]:
import time


class PersonThread(threading.Thread):
    def __init__(self, n):
        super().__init__()
        self.n = n

    def run(self):
        th = threading.current_thread()
        print(f"{th.name=}")

        for i in range(self.n):
            time.sleep(1)
            print(f"slept {i}")


th = PersonThread(10)
th.start()
# th.join()

th.name='Thread-4'


slept 0
slept 1
slept 2
slept 3


In [13]:
th.is_alive()

True

slept 4
slept 5
slept 6
slept 7
slept 8
slept 9


In [15]:
%%time

N = 100_000_000


def counter(a, b):
    while a < b:
        a += 1


counter(0, N)

CPU times: user 13.3 s, sys: 0 ns, total: 13.3 s
Wall time: 14.4 s


In [17]:
%%time

N = 100_000_000
N_THREADS = 8
SIZE = N // N_THREADS


def counter(a, b):
    th = threading.current_thread()
    print(f"{th.name=}, {th.native_id=}")

    while a < b:
        a += 1


threads = [
    threading.Thread(
        target=counter,
        name=f"count-{i}",
        args=(i * SIZE, (i + 1) * SIZE),
    )
    for i in range(N_THREADS)
]

for th in threads:
    th.start()

for th in threads:
    th.join()

th.name='count-0', th.native_id=3321
th.name='count-1', th.native_id=3322
th.name='count-2', th.native_id=3323
th.name='count-3', th.native_id=3324
th.name='count-4', th.native_id=3325
th.name='count-5', th.native_id=3326
th.name='count-6', th.native_id=3327
th.name='count-7', th.native_id=3328
CPU times: user 20.3 s, sys: 1.45 s, total: 21.8 s
Wall time: 21.8 s


In [1]:
from urllib.request import urlopen

In [ ]:
%%time

N = 100
N_THREADS = 8
SIZE = N // N_THREADS

URL = "https://ru.wikipedia.org/wiki/Python"
URLS = [URL] * N


def fetch_url(url):
    resp = urlopen(url)
    return resp


def fetch_batch_urls(urls):
    th = threading.current_thread()
    print(f"{th.name=}, {th.native_id=}, {len(urls)=}")

    for url in urls:
        fetch_url(url)


fetch_batch_urls(URLS)

In [13]:
resp = urlopen(URL)

In [14]:
resp.read().decode("utf-8")

'<!DOCTYPE html>\n<html class="client-nojs" lang="ru" dir="ltr">\n<head>\n<meta charset="UTF-8">\n<title>Python — Википедия</title>\n<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":[",\\t.","\xa0\\t,"],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","январь","февраль","март","апрель","май","июнь","июль","август","сентябрь","октябрь","ноябрь","декабрь"],"wgRequestId":"a77a9dcf-d244-4162-afa1-98440bb2fd8b","wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"Python","wgTitle":"Python","wgCurRevisionId":133677983,"wgRevisionId":133677983,"wgArticleId":2705,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Страницы, использующие устаревший тег source","Википедия:Cite web (не указан язык)","Википедия:Cite web (заменить webcitation-архив: deadlink no)",\n"Страницы, использующие волшебные ссы

In [19]:
%%time

N = 100
N_THREADS = 4
SIZE = N // N_THREADS

URL = "https://ru.wikipedia.org/wiki/Python"
URLS = [URL] * N


def fetch_url(url):
    time.sleep(0.1)


#     resp = urlopen(url)
#     return resp


def fetch_batch_urls(urls):
    th = threading.current_thread()
    print(f"{th.name=}, {th.native_id=}, {len(urls)=}")

    for url in urls:
        fetch_url(url)


def run_exp(n_threads, n):
    size = n // n_threads

    threads = [
        threading.Thread(
            target=fetch_batch_urls,
            name=f"fetch-{i}",
            args=(URLS[i * size : (i + 1) * size],),
        )
        for i in range(n_threads)
    ]

    for th in threads:
        th.start()

    for th in threads:
        th.join()

CPU times: user 23 µs, sys: 1 µs, total: 24 µs
Wall time: 36.7 µs


In [21]:
%%time

run_exp(4, 100)

th.name='fetch-0', th.native_id=3506, len(urls)=25
th.name='fetch-1', th.native_id=3507, len(urls)=25
th.name='fetch-2', th.native_id=3508, len(urls)=25
th.name='fetch-3', th.native_id=3509, len(urls)=25


CPU times: user 18.9 ms, sys: 9.57 ms, total: 28.5 ms
Wall time: 2.69 s


In [28]:
%%time

run_exp(60, 100)

th.name='fetch-0', th.native_id=3916, len(urls)=1
th.name='fetch-1', th.native_id=3917, len(urls)=1
th.name='fetch-2', th.native_id=3918, len(urls)=1
th.name='fetch-3', th.native_id=3919, len(urls)=1
th.name='fetch-4', th.native_id=3920, len(urls)=1
th.name='fetch-5', th.native_id=3921, len(urls)=1
th.name='fetch-6', th.native_id=3922, len(urls)=1
th.name='fetch-7', th.native_id=3923, len(urls)=1
th.name='fetch-8', th.native_id=3924, len(urls)=1
th.name='fetch-9', th.native_id=3925, len(urls)=1
th.name='fetch-10', th.native_id=3926, len(urls)=1
th.name='fetch-11', th.native_id=3927, len(urls)=1
th.name='fetch-12', th.native_id=3928, len(urls)=1
th.name='fetch-13', th.native_id=3929, len(urls)=1
th.name='fetch-14', th.native_id=3930, len(urls)=1
th.name='fetch-15', th.native_id=3931, len(urls)=1
th.name='fetch-16', th.native_id=3932, len(urls)=1
th.name='fetch-17', th.native_id=3933, len(urls)=1
th.name='fetch-18', th.native_id=3934, len(urls)=1
th.name='fetch-19', th.native_id=3935, le

In [65]:
%%time

N = 100
N_THREADS = 10
SIZE = N // N_THREADS

URL = "https://ru.wikipedia.org/wiki/Python"
URLS = [URL] * N


def fetch_url(url):
    resp = urlopen(url)
    return resp


def fetch_batch_urls(urls):
    th = threading.current_thread()
    print(f"{th.name=}, {th.native_id=}, {len(urls)=}")

    for url in urls:
        fetch_url(url)


def run_exp(n_threads, n):
    size = n // n_threads

    threads = [
        threading.Thread(
            target=fetch_batch_urls,
            name=f"fetch-{i}",
            args=(URLS[i * size : (i + 1) * size],),
        )
        for i in range(n_threads)
    ]

    for th in threads:
        th.start()

    for th in threads:
        th.join()


run_exp(N_THREADS, N)

th.name='fetch-0', th.native_id=21657980, len(urls)=10
th.name='fetch-1', th.native_id=21657981, len(urls)=10
th.name='fetch-2', th.native_id=21657986, len(urls)=10
th.name='fetch-3', th.native_id=21657988, len(urls)=10
th.name='fetch-4', th.native_id=21657989, len(urls)=10
th.name='fetch-5', th.native_id=21657994, len(urls)=10
th.name='fetch-6', th.native_id=21657995, len(urls)=10
th.name='fetch-7', th.native_id=21657996, len(urls)=10th.name='fetch-8', th.native_id=21657997, len(urls)=10

th.name='fetch-9', th.native_id=21657998, len(urls)=10
CPU times: user 973 ms, sys: 114 ms, total: 1.09 s
Wall time: 2.09 s


In [69]:
%%time

N = 1000
counter = [0]


def count_operations(a, b):
    for i in range(a, b):
        # actions
        counter[0] += 1


count_operations(0, N)

print(counter, counter[0] == N)

[1000] True
CPU times: user 310 µs, sys: 72 µs, total: 382 µs
Wall time: 358 µs


In [29]:
%%time

N = 1000
N_THREADS = 100
SIZE = N // N_THREADS

counter = [0]
lock = threading.Lock()


def count_operations(a, b, lock):
    for i in range(a, b):
        # actions
        lock.acquire()

        cnt = counter[0]
        cnt += 1
        for _ in range(i):
            pass
        counter[0] = cnt

        lock.release()


def run_exp(n_threads, n, lock):
    size = n // n_threads

    threads = [
        threading.Thread(
            target=count_operations,
            name=f"count-{i}",
            args=(i * size, (i + 1) * size, lock),
        )
        for i in range(n_threads)
    ]

    for th in threads:
        th.start()

    for th in threads:
        th.join()


run_exp(N_THREADS, N, lock)


print(counter, counter[0] == N)

[1000] True
CPU times: user 0 ns, sys: 47.2 ms, total: 47.2 ms
Wall time: 114 ms


In [30]:
import sys

In [34]:
sys.getswitchinterval()

0.0

In [33]:
sys.setswitchinterval(0.0000001)

In [89]:
%%time

N = 1000
N_THREADS = 100
SIZE = N // N_THREADS

counter = [0]
lock = threading.Lock()


def count_operations(a, b, lock):
    for i in range(a, b):
        # actions
        with lock:
            cnt = counter[0]
            cnt += 1
            for _ in range(i):
                pass
            counter[0] = cnt


def run_exp(n_threads, n, lock):
    size = n // n_threads

    threads = [
        threading.Thread(
            target=count_operations,
            name=f"count-{i}",
            args=(i * size, (i + 1) * size, lock),
        )
        for i in range(n_threads)
    ]

    for th in threads:
        th.start()

    for th in threads:
        th.join()


run_exp(N_THREADS, N, lock)


print(counter, counter[0] == N)

[1000] True
CPU times: user 35.1 ms, sys: 74.4 ms, total: 110 ms
Wall time: 54.9 ms


In [93]:
%%time

N = 100
N_THREADS = 10
SIZE = N // N_THREADS

URL = "https://ru.wikipedia.org/wiki/Python"
URLS = [URL] * N

sema = threading.Semaphore(5)


def fetch_url(url, sema):
    with sema:
        resp = urlopen(url)
    return resp


def fetch_batch_urls(urls, sema):
    th = threading.current_thread()
    print(f"{th.name=}, {th.native_id=}, {len(urls)=}")

    for url in urls:
        fetch_url(url, sema)


def run_exp(n_threads, n, sema):
    size = n // n_threads

    threads = [
        threading.Thread(
            target=fetch_batch_urls,
            name=f"fetch-{i}",
            args=(URLS[i * size : (i + 1) * size], sema),
        )
        for i in range(n_threads)
    ]

    for th in threads:
        th.start()

    for th in threads:
        th.join()


run_exp(N_THREADS, N, sema)

th.name='fetch-0', th.native_id=21689955, len(urls)=10th.name='fetch-1', th.native_id=21689956, len(urls)=10
th.name='fetch-2', th.native_id=21689957, len(urls)=10th.name='fetch-3', th.native_id=21689958, len(urls)=10
th.name='fetch-4', th.native_id=21689959, len(urls)=10


th.name='fetch-5', th.native_id=21689960, len(urls)=10th.name='fetch-6', th.native_id=21689961, len(urls)=10th.name='fetch-7', th.native_id=21689962, len(urls)=10

th.name='fetch-8', th.native_id=21689963, len(urls)=10th.name='fetch-9', th.native_id=21689967, len(urls)=10


CPU times: user 1.03 s, sys: 764 ms, total: 1.8 s
Wall time: 3.57 s


In [1]:
from queue import Queue, LifoQueue, PriorityQueue

In [105]:
%%time

N = 100
N_THREADS = 5
SIZE = N // N_THREADS

URL = "https://ru.wikipedia.org/wiki/Python"
URLS = [URL] * N

que = Queue(maxsize=100)


def print_except(*args, **kwargs):
    name = threading.current_thread().name
    print(name, args, kwargs)


threading.excepthook = print_except


def fetch_url_worker(que):
    while True:
        url = que.get()
        name = threading.current_thread().name
        if name.endswith("-3"):
            raise Exception("WRONG!!!")

        if url is None:
            que.put(url)
            print(f"STOP THREAD {name=}")
            break

        resp = urlopen(url)
        # process(resp)


def run_exp(n_threads, n, que):
    size = n // n_threads

    threads = [
        threading.Thread(
            target=fetch_url_worker,
            name=f"fetch-{i}",
            args=(que,),
        )
        for i in range(n_threads)
    ]

    for th in threads:
        th.start()

    for url in URLS:
        que.put(url)
    que.put(None)

    for th in threads:
        th.join()


run_exp(N_THREADS, N, que)

print("global end")

fetch-3 (_thread._ExceptHookArgs(exc_type=<class 'Exception'>, exc_value=Exception('WRONG!!!'), exc_traceback=<traceback object at 0x11366d900>, thread=<Thread(fetch-3, started 123145541009408)>),) {}
STOP THREAD name='fetch-1'
STOP THREAD name='fetch-4'
STOP THREAD name='fetch-0'
STOP THREAD name='fetch-2'
global end
CPU times: user 1.01 s, sys: 337 ms, total: 1.35 s
Wall time: 4.72 s


In [106]:
threading.main_thread()

<_MainThread(MainThread, started 4526339584)>